In [41]:
import pandas as pd
import os

In [42]:
## Get and set working directory
path = "/Users/matthewmacrides/Python/Data Sources/4650"
os.chdir(path)
path = os.getcwd()
print(path)

/Users/matthewmacrides/Python/Data Sources/4650


In [43]:
# Load in the Chicago crime data
crime = pd.read_csv('input.csv', index_col=False)
# Print
crime.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,3,10508693,HZ250496,05/03/2016 11:40:00 PM,013XX S SAWYER AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,...,24.0,29.0,08B,1154907.0,1893681.0,2016,05/10/2016 03:56:50 PM,41.864073,-87.706819,"(41.864073157, -87.706818608)"
1,89,10508695,HZ250409,05/03/2016 09:40:00 PM,061XX S DREXEL AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,...,20.0,42.0,08B,1183066.0,1864330.0,2016,05/10/2016 03:56:50 PM,41.782922,-87.604363,"(41.782921527, -87.60436317)"
2,197,10508697,HZ250503,05/03/2016 11:31:00 PM,053XX W CHICAGO AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,...,37.0,25.0,24,1140789.0,1904819.0,2016,05/10/2016 03:56:50 PM,41.894908,-87.758372,"(41.894908283, -87.758371958)"
3,673,10508698,HZ250424,05/03/2016 10:10:00 PM,049XX W FULTON ST,0460,BATTERY,SIMPLE,SIDEWALK,False,...,28.0,25.0,08B,1143223.0,1901475.0,2016,05/10/2016 03:56:50 PM,41.885687,-87.749516,"(41.885686845, -87.749515983)"
4,911,10508699,HZ250455,05/03/2016 10:00:00 PM,003XX N LOTUS AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,...,28.0,25.0,06,1139890.0,1901675.0,2016,05/10/2016 03:56:50 PM,41.886297,-87.761751,"(41.886297242, -87.761750709)"


In [44]:
# Remove data fram 2017 since records aren't until year end
crime = crime[crime.Year != 2017]
# Extract date information to create a 'Month' column and convert into an integer
crime['Month'] = crime['Date'].str.slice(0, 2)
crime[['Month']] = crime[['Month']].apply(pd.to_numeric)
# Drop columns that we wll not include in our data warehouse
crime = crime.drop(columns=['Unnamed: 0', 'Updated On', 'Latitude', 'Longitude', 'X Coordinate', 'Y Coordinate', 'Location', 'Beat', 'IUCR', 'Ward', 'FBI Code', 'Date', 'ID', 'Community Area', 'Description'])
# Sort dataframe by 'Year' then 'Month'
crime = crime.sort_values(by=['Year', 'Month'],  ascending= True)
# Rename a few columns
crime = crime.rename(columns={"Primary Type": "PrimaryType", "Location Description" : "LocationDescription", "Case Number" : "CaseNumber", "District" : "DistrictNo"})

In [34]:
# Number of records our Chicago Crime dataset includes
len(crime.index) 

1445357

In [46]:
# Getting the unique values of the Block, Location Description,
# primary Type, and Case Number columns and putting them into a dimension tables.
# Date is the only dimesnion table manually created in Excel
block = crime.Block.unique()
block = pd.DataFrame(block)
block = block.rename(columns={0: "Block"})
block['BlockKey'] = range(1, len(block) + 1)
block = block[['BlockKey', 'Block']]
locationDescription = crime.LocationDescription.unique()
locationDescription = pd.DataFrame(locationDescription)
locationDescription = locationDescription.rename(columns={0: "LocationDescription"})
locationDescription['LocationDescriptionKey'] = range(1, len(locationDescription) + 1)
locationDescription = locationDescription[['LocationDescriptionKey', 'LocationDescription']]
primaryType = crime.PrimaryType.unique()
primaryType = pd.DataFrame(primaryType)
primaryType = primaryType.rename(columns={0: "PrimaryType"})
primaryType['PrimaryTypeKey'] = range(1, len(primaryType) + 1)
primaryType = primaryType[['PrimaryTypeKey', 'PrimaryType']]
caseNumber = crime.CaseNumber.unique()
caseNumber = pd.DataFrame(caseNumber)
caseNumber = caseNumber.rename(columns={0: "CaseNumber"})
caseNumber['CaseNumberKey'] = range(1, len(caseNumber) + 1)
caseNumber = caseNumber[['CaseNumberKey', 'CaseNumber']]

,BlockKey,Block
0,1,026XX N MC VICKER AVE
1,2,033XX W 38TH PL
2,3,036XX S RHODES AVE
3,4,062XX S ROCKWELL ST
4,5,0000X N AUSTIN BLVD


In [36]:
# Creating Arrest, Domestic, and District Dimension Tables
arrestData = [[0, False], [1, True]]
arrest = pd.DataFrame(arrestData, columns=['ArrestKey', 'Arrest'])
domesticData = [[0, False], [1, True]]
domestic = pd.DataFrame(arrestData, columns=['DomesticKey', 'Domestic'])
districtData = [[1, 'Central'], [2, 'Wentworth'], [3, 'Grand Crossing'], [4, 'South Chicago'], [5, 'Calumet'], [6, 'Gresham'], [7, 'Englewood'], [8, 'Chicago Lawn'], [9, 'Deering'], [10, 'Ogden'], [11, 'Harrison'], [12, 'Near West'], [14, 'Shakespeare'], [15, 'Austin'], [16, 'Jefferson Park'], [17, 'Albany Park'], [18, 'Near North'], [19, 'Town Hall'], [20, 'Lincoln'], [22, 'Morgan Park'], [24, 'Rogers Park'], [25, 'Grand Central']]
district = pd.DataFrame(districtData, columns=['DistrictNo', 'District'])
district['DistrictKey'] = range(1, len(district) + 1)
district = district[['DistrictKey', 'DistrictNo', 'District']]

,DistrictKey,DistrictNo,District
0,1,1,Central
1,2,2,Wentworth
2,3,3,Grand Crossing
3,4,4,South Chicago
4,5,5,Calumet


In [47]:
# Created dimension tables into separate csv files
block.to_csv('Block.csv', index=False)
locationDescription.to_csv('LocationDescription.csv', index=False)
primaryType.to_csv('PrimaryType.csv', index=False)
caseNumber.to_csv('CaseNumber.csv', index=False)
arrest.to_csv('Arrest.csv', index=False)
domestic.to_csv('Domestic.csv', index=False)
district.to_csv('District.csv', index=False)

In [48]:
# Read in the date dimension table
date = pd.read_csv('Date.csv', index_col=False)
# Since the datekeys are already alligned, add the unemployment and poverty rates column to the date dataset - we will merge it into the fact table in the next step
unemployment = pd.read_csv('Unemployment.csv', index_col=False)
poverty = pd.read_csv('Poverty.csv', index_col=False)
date['UnemploymentRate'] = unemployment['UnemploymentRate']
date['PovertyRate'] = poverty['PovertyRate']

In [51]:
# Join the dimension tables with the input (fact) table on their keys
crime = crime.join(caseNumber.set_index('CaseNumber'), on='CaseNumber', lsuffix='left')
crime = crime.join(block.set_index('Block'), on='Block', lsuffix='left')
crime = crime.join(locationDescription.set_index('LocationDescription'), on='LocationDescription', lsuffix='left')
crime = crime.join(primaryType.set_index('PrimaryType'), on='PrimaryType', lsuffix='left')
crime = crime.join(district.set_index('DistrictNo'), on='DistrictNo', lsuffix='left')
crime = crime.join(arrest.set_index('Arrest'), on='Arrest', lsuffix='left')
crime = crime.join(domestic.set_index('Domestic'), on='Domestic', lsuffix='left')
cols = ['Month', 'Year']
crime = crime.join(date.set_index(cols), on=cols, lsuffix='left')
# Drop columns no longer needed in the fact table
crime = crime.drop(columns=['Year', 'Month', 'Arrest', 'Domestic', 'Block', 'LocationDescription', 'PrimaryType', 'DistrictNo', 'District', 'CaseNumber'])

In [52]:
# There were 28 records with a police district of 31 in the input table when that is not an applicable value
# Removing the 28 records with a police district of 31
crime = crime.dropna()
# Now I'm able to convert 'DistrictKey' column from float into integer
crime['DistrictKey'] = crime['DistrictKey'].astype(int)

In [53]:
# Create a column called 'PrimaryKey' within the fact table
crime['PrimaryKey'] = range(1, len(crime) + 1)
# Rearrange columns so the 'PrimaryKey' column is first
crime = crime[['PrimaryKey', 'CaseNumberKey', 'PrimaryTypeKey', 'BlockKey', 'LocationDescriptionKey', 'DistrictKey', 'ArrestKey', 'DomesticKey', 'DateKey', 'UnemploymentRate', 'PovertyRate']]

In [54]:
# Print to show results
crime.head(1000000)

,PrimaryKey,CaseNumberKey,PrimaryTypeKey,BlockKey,LocationDescriptionKey,DistrictKey,ArrestKey,DomesticKey,DateKey,UnemploymentRate,PovertyRate
197,1,1,1,1,1,22,1,0,1,9.9,23.9
198,2,2,1,1,1,22,1,0,1,9.9,23.9
8084,3,3,2,2,2,9,0,1,1,9.9,23.9
13768,4,4,1,3,2,2,0,0,1,9.9,23.9
14448,5,5,3,4,1,8,0,0,1,9.9,23.9
...,...,...,...,...,...,...,...,...,...,...,...
1019071,999996,999965,5,6616,5,3,0,0,41,5.9,20.9
1019072,999997,999966,8,11225,4,21,0,0,41,5.9,20.9
1019073,999998,999967,5,12597,6,22,0,0,41,5.9,20.9
1019074,999999,999968,7,391,2,1,0,0,41,5.9,20.9


In [102]:
# Export fact table into csv
crime.to_csv('Fact.csv')